This file conducts and exploratory analysis on the magnatuneatag dataset. Here are useful links:
- https://github.com/keunwoochoi/magnatagatune-list?tab=readme-ov-file
- https://mirg.city.ac.uk/codeapps/the-magnatagatune-dataset


In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from zipfile import ZipFile 
import os
import librosa
from matplotlib import pyplot as plt

I used `cat mp3.zip.* > mp3_all.zip` to concat the 3 compressed folders from the links above and unzip them below

In [5]:
# # loading the zip file and creating a zip object 
# with ZipFile("/home/mendu/Thesis/data/magnatagatune/mp3_all.zip", 'r') as zObject: 
#     zObject.extractall(path="/home/mendu/Thesis/data/magnatagatune/audio") 

I ran `find audio -type f -name "*.mp3" -exec mv {} audio \;` to get all the .mp3 files into the audio folder and
`find audio -type d -empty -delete` to delete the empty folders

In [2]:
# Checking the no. of audio files
audio_dir = '/home/mendu/Thesis/data/magnatagatune/audio'
audio_list = os.listdir(audio_dir)
audio_list = [file_name.replace('.wav.mp3', '') for file_name in audio_list] # list of audio track names
print('Total number of audio files', len(audio_list))

Total number of audio files 25863


In [3]:
data = pd.read_csv('/home/mendu/Thesis/data/magnatagatune/tag_annotations.csv',sep = "\t" ,index_col=[0])
data['mp3_path'] = [i[2:] for i in data['mp3_path']] 

In [4]:
genre = ['classical', 'clasical', 'classic', 'techno', 'electronic', 'electro', 'electronica', 
 'electric', 'rock', 'indian', 'india', 'opera', 'operatic', 'pop', 'new age', 'dance', 
 'country', 'metal', 'heavy', 'heavy metal']

synonym_dict = {'classical':['classical', 'clasical', 'classic'],
                'electronic': ['electronic', 'electro', 'electronica', 
 'electric'],
                'indian': ['indian', 'india'],
                'opera': ['opera', 'operatic'],
                'metal': ['metal', 'heavy', 'heavy metal']}

genre.append('mp3_path')

data = data[genre]

In [5]:
data

,classical,clasical,classic,techno,electronic,electro,electronica,electric,rock,indian,...,opera,operatic,pop,new age,dance,country,metal,heavy,heavy metal,mp3_path
clip_id,,,,,,,,,,,,,,,,,,,,,
2,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,american_bach_soloists-j_s__bach_solo_cantatas...
6,1,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,american_bach_soloists-j_s__bach_solo_cantatas...
10,1,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,american_bach_soloists-j_s__bach_solo_cantatas...
11,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,american_bach_soloists-j_s__bach_solo_cantatas...
12,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,american_bach_soloists-j_s__bach_solo_cantatas...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58899,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,jacob_heringman-blame_not_my_lute-56-la_bressa...
58906,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,jacob_heringman-blame_not_my_lute-57-lost_is_m...
58907,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,jacob_heringman-blame_not_my_lute-57-lost_is_m...


In [6]:
data['tags'] = data.apply(lambda row: (','.join(row.index[row == 1])).split(','), axis=1)

data = data[['mp3_path', 'tags']]

In [12]:
data_ = data[data['tags'].apply(len) > 0]

In [24]:
isinstance(data.tags[-4:-3], list)


False

In [46]:
for idx, row in data.iterrows(): #iterates over the rows
    new_tags = []
    for tag in row['tags']: #iterates over each element in the aspect_list list
        for key, values in synonym_dict.items(): #iterates over same_tags dictionary
            if tag in values:
                new_tags.append(key)
                break
        else:
            new_tags.append(tag)
    if len(new_tags) > 1 and 'classical' in new_tags: #if that are more than one tags and one of it is instrumental drop instrumental
        new_tags.remove('classical')
    data.at[idx, 'tags'] = list(set(new_tags))

In [25]:
data['tags'][2][0]

'classical'

In [65]:
synonyms = [['beat', 'beats'],
			['chant', 'chanting'],
			['choir', 'choral'],
			['classical', 'clasical', 'classic'],
			['drum', 'drums'],
			['electronic', 'electro', 'electronica', 'electric'],
			['fast', 'fast beat', 'quick'],
			['female', 'female singer', 'female singing', 'female vocals', 'female voice', 'woman', 'woman singing', 'women'],
			['flute', 'flutes'],
			['guitar', 'guitars'],
			['hard', 'hard rock'],
			['harpsichord', 'harpsicord'],
			['heavy', 'heavy metal', 'metal'],
			['horn', 'horns'],
			['indian', 'india'],
			['jazz', 'jazzy'],
			['male', 'male singer', 'male vocal', 'male vocals', 'male voice', 'man', 'man singing', 'men'],
			['no beat', 'no drums'],
			['no singer', 'no singing', 'no vocal','no vocals', 'no voice', 'no voices', 'instrumental'],
			['opera', 'operatic'],
			['orchestra', 'orchestral'],
			['quiet', 'silence'],
			['singer', 'singing'],
			['space', 'spacey'],
			['string', 'strings'],
			['synth', 'synthesizer'],
			['violin', 'violins'],
			['vocal', 'vocals', 'voice', 'voices'],
			['strange', 'weird']]

In [71]:
# List of tags to keep
tags_to_keep = ['classical', 'techno', 'electronic', 'rock', 'indian', 'opera', 'pop', 'new age', 'dance', 'country', 'metal']

def replace_with_first_synonym(tag, synonyms):
    unique_tags = set()  # Initialize a set to store unique tags
    for synonym_list in synonyms:
        for synonym in synonym_list:
            if synonym in tag:
                unique_tags.add(synonym_list[0])  # Add the first word of the synonym list to the set of unique tags
                break  # Break the loop if a synonym is found to avoid duplicates
    return ','.join(tag for tag in unique_tags if tag in tags_to_keep)  # Join the unique tags that are in the list of tags to keep

# Apply the function to each tag in the 'tags' column
data['tags'] = data['tags'].apply(lambda tags: replace_with_first_synonym(tags, synonyms))

# Drop rows with no tags
data = data[data['tags'].str.len() > 0]

In [73]:
data.tags.value_counts()

tags
classical          4018
opera               897
classical,opera     283
opera,classical     177
Name: count, dtype: int64